In [1]:
from modules.import_libs import*
from modules.Airloop_functions import(
    find_road_directories,
    find_district_directory,
    extract_lat_long_from_srt
    )

# All roads in "Formating Data" directory
Data_directory = 'Data' + os.sep + 'Formating Data'

In [2]:
def rename_dir_as_survey_form(district_path,roads):
    """
    Rename directories containing road survey forms based on information extracted from the survey forms.

    This function iterates through the given 'roads' directories within the specified 'district_name' folder,
    searching for survey forms in Excel format. It extracts the road name from the survey form and renames the
    corresponding directory to match the extracted road name. If the directory name does not match the road name
    in the survey form, it is corrected.

    Parameters:
    - district_name (str): The name of the district where the roads are located.
    - roads (list of str): A list of directory paths representing the roads to be processed.
    Note:
    - This function assumes that the survey forms are stored in 'segments' subdirectories within the road directories.
    - The survey form should contain the road name as an entry, which will be used for renaming.
    - Directories that already have correct names as per the survey form will not be renamed.
    """
    count=0
    
    for road in roads:
        survey_forms_list = glob.glob(road + os.sep +'segments'+os.sep+  '*/**/*Survey Form*.xlsx', recursive=True)
        # print(road)
        # for survey_form in survey_forms_list:
        sf_df = pd.read_excel(survey_forms_list[0])
        road_path = os.path.dirname(os.path.dirname(survey_forms_list[0])).split('\segments')[0]
        road_name_from_survey_form = sf_df.iloc[1].to_list()[1].strip()
        road_path_corrected = district_path + os.sep + road_name_from_survey_form
        
        if road_path != road_path_corrected.strip():
            o_name=os.path.basename(road_path)
            n_name=os.path.basename(road_path_corrected)
            print(o_name, ' <<< Changed To >>> ', n_name)
            count +=1
            Path(road_path).rename(road_path_corrected.strip())               
    print(f'Renamed total {count} roads out of {len(roads)}... ')

In [3]:
def amend_sForm_and_RenameFiles(roads_list):
    """
    Rename segment directories and update survey form information within each segment.

    This function takes a list of road directories and iterates through the segments within each road. It renames
    the segment directories and associated files, ensuring proper naming conventions are followed. Additionally,
    it updates specific information within the survey forms found in each segment, such as start and end points,
    segment length, and segment identifier.

    Parameters:
    - roads_list (list of str): A list of directory paths representing the roads containing segments to be processed.
    
    Note:
    - This function assumes a specific naming convention for segment directories and associated files.
    - Segment directories are expected to have names like "Segment 01", "Segment 02", etc.
    - Associated files include coordinates files, .SRT files, survey forms, crack area files, crack count files,
      and video files, with naming conventions such as "DJI_0001.csv", "DJI_0001.SRT", "Survey Form Segment 01.xlsx",
      "DJI_0001_2.csv", "DJI_0001.MOV", etc.
    - Survey form information is updated within the Excel files.
    """
    segments_list = []
    segments_list_old = []
    segments_list_old1 = []
    
    # Renaming segment directories
    for rr1 in range(len(roads_list)):
        segments_list_old = glob.glob(roads_list[rr1] + os.sep + 'segments' + os.sep + '*')
        total_number_of_segments = len(segments_list_old)
        sno = 0
        for rr2 in range(len(segments_list_old)):
            old_no = segments_list_old[rr2].split(os.sep)[-1].split(' ')[1]
            doesNotContainAlpha = not(any(char.isalpha() for char in old_no))
            print(segments_list_old[rr2])
            if doesNotContainAlpha :
                old_no = int(old_no)
            new_no = 0
            if total_number_of_segments in range(1,100):
                if old_no in range(1, 10):
                    new_no = '0' + str(old_no)
                else:
                    new_no = old_no
                splits = segments_list_old[rr2].split(os.sep)
                splits[-1] = splits[-1].split(' ')[0] + ' ' + str(new_no)
                path_new = ''
            elif total_number_of_segments > 99:
                if old_no in range(1, 10):
                    new_no = '00' + str(old_no)
                elif old_no in range(10, 100):
                    new_no = '0' + str(old_no)
                else:
                    new_no = old_no

            splits = segments_list_old[rr2].split(os.sep)
            splits[-1] = splits[-1].split(' ')[0] + ' ' + str(new_no)
            path_new = ''
            for rr3 in range(len(splits)):
                if path_new == '':
                    path_new = splits[rr3]
                else:
                    path_new = path_new + os.sep + splits[rr3]   
            segments_list.append(path_new)
            segments_list_old1.append(segments_list_old[rr2])   
       
    for itrr4 in range(len(segments_list)):
        src2 = segments_list_old1[itrr4]
        dst2 = segments_list[itrr4]
        os.rename(src2, dst2)
        
    # Files Renaming
    srt_file = ''
    survey_form_file = ''
    crack_count_file = ''
    vid_file = ''

    srt_name = ''
    survey_form_name = ''
    crack_count_name = ''
    vid_name = ''

    for itra in range(len(roads_list)):
        segments = glob.glob(roads_list[itra] +os.sep+ 'segments' + os.sep + '*')
        for itrb in range(len(segments)):
            seg_path = segments[itrb]
            seg_files = glob.glob(segments[itrb] + os.sep + '*')
            seg_no = segments[itrb].split(os.sep)[-1].split(' ')[1]
            
            
            
            
            
            
            for itrc in range(len(seg_files)):
                if '.SRT' in seg_files[itrc]:
                    srt_file = seg_files[itrc]
                    srt_name = seg_path + os.sep + 'DJI_00' + str(seg_no) + '.SRT'
                    if not os.path.exists(srt_name):
                        os.rename(srt_file, srt_name)
                    
                                    
                elif '.KML' in seg_files[itrc] or '.kml' in seg_files[itrc]:
                    kml_file = seg_files[itrc]
                    kml_name = seg_path + os.sep + 'DJI_00' + str(seg_no) + '.KML'
                    if not os.path.exists(kml_name):
                        os.rename(kml_file, kml_name)
                        
                elif '.xlsx' in seg_files[itrc] and 'Survey Form' in seg_files[itrc]:
                    survey_form_file = seg_files[itrc]
                    survey_form_name = seg_path + os.sep + 'Survey Form Segment ' + str(seg_no) + '.xlsx'
                    if not os.path.exists(survey_form_name):
                        Path(survey_form_file).rename(survey_form_name)
                        
                elif '.csv' in seg_files[itrc] and '_2.csv' not in seg_files[itrc] and 'ind_cracks' not in seg_files[itrc]:
                    crack_count_file = seg_files[itrc]
                    crack_count_name = seg_path + os.sep + 'DJI_00' + str(seg_no) + '.csv'
                    if not os.path.exists(crack_count_name):
                        os.rename(crack_count_file, crack_count_name)
                    
                elif '.csv' in seg_files[itrc] and 'ind_cracks' in seg_files[itrc]:
                    ind_count_file = seg_files[itrc]
                    ind_crack_count_name = seg_path + os.sep + 'ind_cracks_00' + str(seg_no) + '.csv'
                    if not os.path.exists(ind_crack_count_name):
                        os.rename(ind_count_file, ind_crack_count_name)
                    
                elif ('.MOV' in seg_files[itrc] or '.MP4' in seg_files[itrc] or '.mp4' in seg_files[itrc] )and 'detected' not in seg_files[itrc]:                        
                    vid_file = seg_files[itrc]
                    if '.MP4' in vid_file:
                        v_extension = '.MP4'
                    elif '.MOV' in vid_file:
                        v_extension = '.MOV'
                    elif '.mp4' in vid_file:
                        v_extension = '.mp4'
                    vid_name = seg_path + os.sep + 'DJI_00' + str(seg_no) + v_extension
                    if not os.path.exists(vid_name):
                        os.rename(vid_file, vid_name)
            
            
            s_points,e_points = extract_lat_long_from_srt(srt_name)      
            
            s_points_str = f"{s_points[0]}, {s_points[1]}"
            e_points_str = f"{e_points[0]}, {e_points[1]}"
            
            # changing survey form text
            wb = openpyxl.load_workbook(survey_form_name) 
            sheet = wb.active
            
            for rowNo in range(1,16):
                cellNo = 'A'+str(rowNo)
                cellValue = sheet[cellNo].value
                sheet[cellNo] = cellValue.strip()
            
            
            # updating correct start and end points
         
            sheet['B13'] = s_points_str
            sheet['B14'] = e_points_str
            
            
            # updating correct segment length
            seg_len = sheet['B9'].value
            if 'km' in seg_len:
                seg_len = seg_len.strip().split('km')[0]
                seg_len = str(int(float(seg_len) * 1000)) + 'M'
                
            if 'KM' in seg_len:
                seg_len = seg_len.strip().split('KM')[0]
                seg_len = str(int(float(seg_len) * 1000)) + 'M'
                
            sheet['B9'] = seg_len
            road_name = sheet['B3'].value
            newroadname = ' '.join(road_name.split())
            newroadname = newroadname.strip()
            
            # Create a translation table to remove the specified characters
            chars_to_remove = '[<>:"/\\|?*%-_^!.]'
            translation_table = str.maketrans('', '', chars_to_remove)

            # Translate the text using the translation table
            newroadname = newroadname.translate(translation_table)

             
            sheet['B3'] = newroadname
            sheet['B2'] = 'DJI_00' + str(seg_no)
            sheet['B8'] = str(seg_no)
            wb.save(survey_form_name)

    

In [4]:
def main():
    roads = [i for i in (glob.glob(Data_directory + os.sep + '*')) if os.path.isdir(i)]
    amend_sForm_and_RenameFiles(roads)
    rename_dir_as_survey_form(Data_directory,roads) 
    print('done!')
if __name__ == "__main__":
    main()

Data\Formating Data\Amankot Rd\segments\Segment 01
Data\Formating Data\Amankot Rd\segments\Segment 02
Data\Formating Data\Amankot Rd\segments\Segment 03
Data\Formating Data\Amankot Rd\segments\Segment 04
Data\Formating Data\Amankot Rd\segments\Segment 05
Data\Formating Data\Amankot Rd\segments\Segment 06
Data\Formating Data\Amankot Rd\segments\Segment 07
Data\Formating Data\Amankot Rd\segments\Segment 08
Data\Formating Data\Amankot Rd\segments\Segment 09
Data\Formating Data\Amankot Rd\segments\Segment 10
Data\Formating Data\Amankot Rd\segments\Segment 11A
Data\Formating Data\Amankot Rd\segments\Segment 11B
Data\Formating Data\Amankot Rd\segments\Segment 12
Data\Formating Data\Balo Haryan RD\segments\Segment 01
Data\Formating Data\Balo Haryan RD\segments\Segment 02
Data\Formating Data\Balo Haryan RD\segments\Segment 03
Data\Formating Data\Balo Haryan RD\segments\Segment 04
Data\Formating Data\Bamkhel Marghuz Rd\segments\Segment 01
Data\Formating Data\Bamkhel Marghuz Rd\segments\Segment 